In [4]:
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
from tkinter import Toplevel

In [5]:
class RasterViewer:
    def __init__(self, root):
        self.root = root
        self.root.title("Raster Viewer")
        self.image = None
        self.image_width = 0
        self.image_height = 0
        self.original_image_width = 0  
        self.original_image_height = 0 
        self.current_band_combination = (0, 1, 2)  # Default RGB bands
        self.current_zoom = 1.0
        self.zoomed_image = None
        self.canvas = tk.Canvas(self.root, cursor="cross")
        self.canvas.pack(fill=tk.BOTH, expand=True)
        self.canvas.bind("<MouseWheel>", self.zoom)
        self.canvas.bind("<ButtonPress-2>", self.start_pan)
        self.canvas.bind("<B2-Motion>", self.pan)

        self.menu_bar = tk.Menu(self.root)
        self.root.config(menu=self.menu_bar)

        self.file_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="File", menu=self.file_menu)
        self.file_menu.add_command(label="Open Image", command=self.open_image)
        self.file_menu.add_command(label="Open Shapefile", command=self.open_shapefile)
        self.file_menu.add_separator()
        self.file_menu.add_command(label="Exit", command=self.root.destroy)

        self.band_combinations = [
            ("RGB", (0, 1, 2)),
            ("Red", (0,)),
            ("Green", (1,)),
            ("Blue", (2,)),
            ("NIR-RGB", (3, 0, 1)),
        ]
        self.band_combination_var = tk.StringVar(root)
        self.band_combination_var.set("RGB")
        self.band_combination_menu = tk.OptionMenu(root, self.band_combination_var, *map(lambda x: x[0], self.band_combinations), command=self.change_band_combination)
        self.band_combination_menu.pack()

        self.left_button = tk.Button(self.root, text="Left", command=lambda: self.pan_button("left"))
        self.left_button.pack(side=tk.LEFT)

        self.right_button = tk.Button(self.root, text="Right", command=lambda: self.pan_button("right"))  # Remove "0"
        self.right_button.pack(side=tk.LEFT)

        self.mean_button = tk.Button(self.root, text="Display Mean", command=self.display_mean)
        self.mean_button.pack(side=tk.LEFT)

        self.std_deviation_button = tk.Button(self.root, text="Display Standard Deviation", command=self.display_std_deviation)
        self.std_deviation_button.pack(side=tk.LEFT)

        self.histogram_button = tk.Button(self.root, text="Display Histogram", command=self.display_histogram)
        self.histogram_button.pack(side=tk.LEFT)

    def open_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg;*.bmp;*.gif")])

        if file_path:
            self.image = Image.open(file_path)
            self.image_width, self.image_height = self.image.size
            self.original_image_width, self.original_image_height = self.image.size 
            self.display_image()

    def open_shapefile(self):
        shapefile_path = filedialog.askopenfilename(filetypes=[("Shapefile files", "*.shp")])

        if shapefile_path:
            self.shapefile = gpd.read_file(shapefile_path)
            self.attribute_data = None
            self.update_attribute_dropdown()

    def update_attribute_dropdown(self):
        if hasattr(self, 'shapefile'):
            attributes = [col for col, dtype in self.shapefile.dtypes.items() if dtype not in ['geometry', 'object']]
            self.attribute_var = tk.StringVar(self.root)
            self.attribute_var.set("")            
            attribute_menu = tk.OptionMenu(self.root, self.attribute_var, *attributes, command=self.plot_chart)
            attribute_menu.pack()

    def plot_chart(self, event):
        selected_attribute = self.attribute_var.get()
        if selected_attribute and hasattr(self, 'shapefile'):
            attribute_data = [feature.get(selected_attribute, None) for feature in self.shapefile.iterfeatures()]

            # Filter out None values
            valid_data_indices = [i for i, value in enumerate(attribute_data) if value is not None]

            if not valid_data_indices:
                print(f"Attribute '{selected_attribute}' not found in any feature.")
                return

            unique_labels = list(set(value for i, value in enumerate(attribute_data) if i in valid_data_indices and not isinstance(value, dict)))

            plt.clf()
            if isinstance(attribute_data[valid_data_indices[0]], (int, float)):
                
                plt.hist([attribute_data[i] for i in valid_data_indices], bins=20, color='blue', alpha=0.7)
                plt.title(f"Histogram for {selected_attribute}")
                plt.xlabel(selected_attribute)
                plt.ylabel("Frequency")
            else:
                # Categorical data, create a bar chart
                label_indices = {f"Index {i}": i for i in valid_data_indices}
                plt.bar([label_indices[f"Index {i}"] for i in valid_data_indices], [attribute_data[i] for i in valid_data_indices], tick_label=unique_labels, color='green', alpha=0.7)
                plt.title(f"Bar Chart for {selected_attribute}")
                plt.xlabel("Feature Index")
                plt.ylabel("Count")

            canvas = FigureCanvasTkAgg(plt.gcf(), master=self.root)
            canvas.draw()
            canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

            self.root.update()

    def display_image(self):
        self.canvas.delete("all")
        self.canvas.config(scrollregion=(0, 0, self.original_image_width, self.original_image_height))

        img_array = np.asarray(self.image)
        bands = [img_array[:, :, i] for i in self.current_band_combination]

        if len(bands) == 1:
            mode = 'L'  # Set mode to luminance for single-band images
            merged_array = bands[0]
        else:
            mode = "RGB"
            merged_array = np.stack(bands, axis=-1)  # Stack along the third axis for RGB

        # Convert NumPy array to PIL Image
        self.zoomed_image = Image.fromarray(merged_array, mode=mode)

        self.tk_image = ImageTk.PhotoImage(self.zoomed_image)
        self.canvas.create_image(0, 0, image=self.tk_image, anchor=tk.NW, tags="image_tag")
        print("Displaying Mean")


    def display_mean(self):
        if self.image:
            
            bands = [np.asarray(self.image)[:, :, i] for i in self.current_band_combination]

            band_means = [np.mean(band) for band in bands]

            self.display_stats_in_window(f"Band Means: {band_means}")
        else:
            print("No image loaded.")

    def display_std_deviation(self):
        if self.image:
            # Use NumPy array for further processing
            bands = [np.asarray(self.image)[:, :, i] for i in self.current_band_combination]

            # Calculate standard deviation for each band
            band_std_devs = [np.std(band) for band in bands]

            # Display standard deviation in a new window
            self.display_stats_in_window(f"Band Standard Deviations: {band_std_devs}")
        else:
            print("No image loaded.")

    def display_histogram(self):
        if self.image:
            # Close the existing histogram window if it exists
            if hasattr(self, 'histogram_window'):
                self.histogram_window.destroy()

            # Create a new Toplevel window
            self.histogram_window = Toplevel(self.root)
            self.histogram_window.title("Histogram")

            # Use NumPy array for further processing
            bands = [np.asarray(self.image)[:, :, i] for i in self.current_band_combination]

            # Calculate histogram for each band
            for i, band in enumerate(bands):
                hist, bin_edges = np.histogram(band, bins=256, range=[0, 256])
                self.plot_histogram(hist, bin_edges, f'Histogram for Band {i+1}')

            print("Displaying Histogram")
        else:
            print("No image loaded.")

    # New method to plot histogram in the separate window
    def plot_histogram(self, hist, bin_edges, title):
    # Create a Tkinter canvas to embed the plot in the Toplevel window
        canvas = FigureCanvasTkAgg(plt.figure(), master=self.histogram_window)

        plt.bar(bin_edges[:-1], hist, width=1)
        plt.title(title)
        plt.xlabel('Pixel Intensity')
        plt.ylabel('Frequency')

        canvas.draw()
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
    def zoom(self, event):
        factor = 1.2 if event.delta > 0 else 0.8
        self.current_zoom *= factor
        new_width = int(self.original_image_width * self.current_zoom)
        new_height = int(self.original_image_height * self.current_zoom)

        self.zoomed_image = self.zoomed_image.resize((new_width, new_height), Image.LANCZOS)
        self.tk_image = ImageTk.PhotoImage(self.zoomed_image)
        self.canvas.config(scrollregion=(0, 0, new_width, new_height))
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)

    def start_pan(self, event):
        self.start_x = event.x_root
        self.start_y = event.y_root

    def pan(self, event):
        delta_x = self.start_x - event.x_root
        delta_y = self.start_y - event.y_root
        self.start_x = event.x_root
        self.start_y = event.y_root

        self.canvas.scan_mark(0, 0)  # Reset the scan mark
        self.canvas.scan_dragto(-delta_x, -delta_y, gain=1)

        # Adjust scroll region if needed
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        self.canvas.config(scrollregion=(
            max(0, self.canvas.canvasx(0)),
            max(0, self.canvas.canvasy(0)),
            min(self.original_image_width, self.canvas.canvasx(canvas_width)),
            min(self.original_image_height, self.canvas.canvasy(canvas_height))
        ))
        print("Panning: Scrolling region after adjustment:", self.canvas.cget("scrollregion"))
        
    def calculate_pan_delta(self, direction):
        """Calculates the delta values for panning based on direction and image dimensions."""

        pan_distance_factor = 0.1  # Adjust this factor to control panning distance
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()

        if direction == "left":
            delta_x = -pan_distance_factor * canvas_width  # Move left by a fraction of canvas width
            delta_y = 0
        elif direction == "right":
            delta_x = pan_distance_factor * canvas_width  # Move right by a fraction of canvas width
            delta_y = 0
        # Add elif statements for other directions (e.g., up, down) if needed
        else:
            raise ValueError("Invalid direction")
            
        return delta_x, delta_y

    def pan_button(self, direction): 
        delta_x, delta_y = self.calculate_pan_delta(direction)

        # Use NumPy array for efficient cropping and panning
        img_array = np.asarray(self.image)
        cropped_array = img_array[max(0, int(delta_y)):min(self.image_height, int(self.image_height + delta_y)),
                                  max(0, int(delta_x)):min(self.image_width, int(self.image_width + delta_x))]

        # Convert the cropped array back to an image
        self.image = Image.fromarray(cropped_array)

        self.display_image()
        self.adjust_scroll_region_if_needed()

    def change_band_combination(self, event):
        selected_combination = next(filter(lambda x: x[0] == self.band_combination_var.get(), self.band_combinations), None)
        if selected_combination:
            self.current_band_combination = selected_combination[1]
            self.display_image()
            
    def adjust_scroll_region_if_needed(self):
        """Adjusts the scroll region if the visible area extends beyond the image boundaries."""

        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        current_scrollregion = self.canvas.cget("scrollregion")

        # Extract current scroll region coordinates
        current_x1, current_y1, current_x2, current_y2 = map(int, current_scrollregion.split())

        # Calculate adjusted coordinates based on canvas dimensions and image boundaries
        adjusted_x1 = max(0, current_x1)
        adjusted_y1 = max(0, current_y1)
        adjusted_x2 = min(self.original_image_width, current_x2)
        adjusted_y2 = min(self.original_image_height, current_y2)

        # Update scroll region if necessary
        if (current_x1, current_y1, current_x2, current_y2) != (adjusted_x1, adjusted_y1, adjusted_x2, adjusted_y2):
            self.canvas.config(scrollregion=(adjusted_x1, adjusted_y1, adjusted_x2, adjusted_y2))

In [6]:
def main():
    root = tk.Tk()
    raster_viewer = RasterViewer(root)
    root.geometry("800x600")
    root.mainloop()

if __name__ == "__main__":
    main()

Displaying Mean
Displaying Mean
Displaying Mean
Displaying Mean
Displaying Mean
Displaying Mean
Displaying Mean
